In [79]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [80]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [81]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32

In [82]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [83]:
imagePaths = sorted(list(paths.list_images("Dataset")))
random.seed(42)
random.shuffle(imagePaths)

In [84]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    label = 1 if label == "triple_person" else 0
    labels.append(label)

In [101]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{0: 948, 1: 63}

In [85]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [86]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [87]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [88]:
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [89]:
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/25
23/23 [==============================] - 7s 200ms/step - loss: 0.2559 - accuracy: 0.9022 - val_loss: 0.1653 - val_accuracy: 0.9565
Epoch 2/25
23/23 [==============================] - 4s 178ms/step - loss: 0.2217 - accuracy: 0.9311 - val_loss: 0.1777 - val_accuracy: 0.9565
Epoch 3/25
23/23 [==============================] - 4s 179ms/step - loss: 0.2144 - accuracy: 0.9298 - val_loss: 0.1977 - val_accuracy: 0.9565
Epoch 4/25
23/23 [==============================] - 4s 189ms/step - loss: 0.2117 - accuracy: 0.9339 - val_loss: 0.1839 - val_accuracy: 0.9565
Epoch 5/25
23/23 [==============================] - 4s 173ms/step - loss: 0.2191 - accuracy: 0.9311 - val_loss: 0.1686 - val_accuracy: 0.9565
Epoch 6/25
23/23 [==============================] - 4s 187ms/step - loss: 0.2044 - accuracy: 0.9339 - val_loss: 0.1804 - val_accuracy: 0.9565
Epoch 7/25
23/23 [==============================] - 4s 167ms/step - loss: 0.2098 - accuracy: 0.9298 - val_loss: 0.1694 - 

In [90]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Santa/Not Santa")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [96]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(253, 2)


In [98]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=["triple_rider","double rider"]))

              precision    recall  f1-score   support

triple_rider       0.96      1.00      0.98       242
double rider       0.00      0.00      0.00        11

    accuracy                           0.96       253
   macro avg       0.48      0.50      0.49       253
weighted avg       0.91      0.96      0.94       253



/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,